# Generate Model

## Common Steps

### Import neccesary packages (install them if not installed)

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot
import os
import sys
import struct
from datetime import date
import numpy as np

### Import dataset

In [ ]:
ds = tfds.load('german_credit_numeric', split='train',as_supervised=True)

tamano_lote = 1000

elems = ds.batch(tamano_lote)
lote_entrenamiento = None
for elem in elems:
    lote_entrenamiento = elem
    break

# We can also print it, in case we need some information and/or context about it
# print(lote_entrenamiento)

### Set loss function, optimizer and metrics

In [ ]:
fn_perdida = tf.keras.losses.BinaryCrossentropy()
optimizador = tf.keras.optimizers.SGD(learning_rate=0.01)
metrica = tf.keras.metrics.Precision()

### Create sequential model

#### Reasonably sized model

In [ ]:
tamano_entrada = 24
h0_size = 5
h1_size = 3


model = tf.keras.models.Sequential(name= "MySampleModel")
model.add( tf.keras.layers.InputLayer((tamano_entrada,)))
# Here we add N layers to the model.
model.add(tf.keras.layers.Dense(units = h0_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = h1_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = 1, activation="sigmoid"))

# Build (could also show and plot) model
model.build()
# print(model.summary())
# tf.keras.utils.plot_model(model, 'MySampleModel.png')

# And compile the model
model.compile(loss=fn_perdida, optimizer=optimizador, metrics=metrica)

#### Overdimensioned model
This model is just for trying to stress test the already very optimized BLAS routines. It does not produce any meaningful output, as the easiest way of making the network work is just >0.5 every single input, but it gets the job done in terms of codegen.

This should be also quite useful on showing the benefits of pruning on large networks.

In [ ]:
tamano_entrada = 24
h0_size = 500
h1_size = 800
h2_size = 1000
h3_size = 1200
h4_size = 600
h5_size = 400
h6_size = 200
h7_size = 100
h8_size = 50
h9_size = 1

model = tf.keras.models.Sequential(name= "MySampleModel")
model.add( tf.keras.layers.InputLayer((tamano_entrada,)))
# Here we add N layers to the model.
model.add(tf.keras.layers.Dense(units = h0_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = h1_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = h2_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = h3_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = h4_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = h5_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = h6_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = h7_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = h8_size, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units = h9_size, activation="sigmoid"))

# Build (could also show and plot) model
model.build()
# print(model.summary())
# tf.keras.utils.plot_model(model, 'MySampleModel.png')

# And compile the model
model.compile(loss=fn_perdida, optimizer=optimizador, metrics=metrica)

### Training the model

In [ ]:
num_epochs =  500

history = model.fit(x=lote_entrenamiento[0], y = lote_entrenamiento[1], batch_size = 20, epochs=num_epochs)
history.history

### Obtaining model metrics

In [ ]:
_, precission = model.evaluate(lote_entrenamiento[0], lote_entrenamiento[1], verbose=0)
print('\nBaseline test accuracy: ', precission)

### Saving model to a file

In [ ]:
# Create file and save it
keras_orig_file_path = os.path.join(os.getcwd(), 'MySampleModel.h5')
tf.keras.models.save_model(model, keras_orig_file_path, include_optimizer=False)
print('Saved baseline model to:', keras_orig_file_path)

## (OPTIONAL) Prune the model

### Fine tuning pre-trained model with pruning

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 20
sparse_epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set.

# num_images = tamano_lote
end_step = np.ceil(tamano_lote/batch_size).astype(np.int32) * sparse_epochs

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.00,
                                                               final_sparsity=0.95,
                                                               begin_step=0,
                                                               end_step=end_step)
}

# clone the model. If we do not do this, the original model will be altered too
model_for_pruning = tf.keras.models.clone_model(model)

model_for_pruning = prune_low_magnitude(model_for_pruning, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer=optimizador, # 'adam',
              loss=fn_perdida, # tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=metrica) #['accuracy'])

model_for_pruning.summary()


### Executing pruning process

In [ ]:
num_epochs =  500

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
]

history = model_for_pruning.fit(lote_entrenamiento[0], lote_entrenamiento[1], batch_size=batch_size, epochs=num_epochs, validation_split=validation_split, callbacks=callbacks)
history.history

### Obtaining pruned model metrics

In [ ]:
_, pruned_precission = model_for_pruning.evaluate(lote_entrenamiento[0], lote_entrenamiento[1], verbose=0)
print('\nPruned model test accuracy: ', pruned_precission)

### Compare results

In [ ]:
# Get results for the dense network
temp_results = model.predict(lote_entrenamiento[0])
# print(temp_results)

print("\n\n\n--- TOTAL DENSE ---")
print(sum(val >= 0.5 for val in temp_results))


# And do the same for the pruned one
temp_results = model_for_pruning.predict(lote_entrenamiento[0])
# print(temp_results)

print("\n\n\n--- TOTAL PRUNED ---")
print(sum(val >= 0.5 for val in temp_results))

## Preparations before obtaining C code

### Common functions

In [ ]:
# https://docs.python.org/3/library/struct.html
FLOAT_BE = ">f"
FLOAT_LE = "<f"
DOUBLE_BE = ">d"
DOUBLE_LE = "<d"

def np_value_to_hex(value, byte_format):
    return bytearray(struct.pack(byte_format, value)).hex()

# byte_format is target format for output
def np_array_to_hex(array, byte_format):
    return map(
        lambda layer: list(
            map(lambda v: np_value_to_hex(v, byte_format), layer)
        ),
        array,
    )

### (EXTRA) Some useful functions not used now, but helpful for debugging and development

In [ ]:
# Obtain values for WEIGHTS and BIAS in Big-Endian (human) format, in hex. This is only useful for debugging purposes, as machines usually are LE
for idx,layer in enumerate(model.layers,1):
    print(f"LAYER{idx}_WEIGHTS:")
    print("\n".join(",".join(layer) for layer in np_array_to_hex(layer.get_weights()[0], FLOAT_BE)))
    print(f"LAYER{idx}_BIAS:")
    print("\n".join(",".join(layer) for layer in np_array_to_hex([layer.get_weights()[1]], FLOAT_BE)))

### Export input set to text file, in order to feed it to the neural network

In [ ]:
# Get original stdout here, in order to print to a file
my_stdout = sys.stdout

with open('input.txt', 'w') as f:
    sys.stdout = f
    # Print dims to beginning of file
    print(len(lote_entrenamiento[0]))
    # and data to the rest of it
    for arr in lote_entrenamiento[0]:
        print(*arr.numpy().tolist(), sep=',')
    # Finally restore stdout
    sys.stdout = my_stdout

## Finally generate C code

### Dense code generation
Here we are generating C code for use with OpenBLAS. In this case, AUR package `openblas-lapack` is installed on top of a fully up-to-date Arch Linux 64 bit installation. This not only gives easy and useful library and header file integration into the system, but also allows us to fine tune compilation options, modifying `CFLAGS` and `CXXFLAGS` on `/etc/makepkg.conf`. On ubuntu `libopenblas-dev` does the trick.

We need to have every file into the same folder. That means, our generated `.c` file, along with `common.{c,h}`.

For compilation, we specify our desired flags in the command `gcc -march=native -O3 -s *.c -o dense.out -lm -lcblas`.

If we are on Ubuntu, we need to change `-lcblas` by `-lopenblas`.

Before we finish the previous step, we need to generate the C code for compiling. This can be done by executing the code below.
We can change `MY_MODEL` and `MAP_ALGORYTHM`, but the only one useful for changing here is `MY_MODEL`, as the latter one is the only transfer function implemented in this POC.

In [ ]:
MY_MODEL = model
MAP_ALGORITHM = "sigmoid__fp32"

header = (f"""/****************************************************************************
 * Copyright (C) {date.today().year} by Alonso Rodriguez                                   *
 *                                                                          *
 * This file is an auto-generated test.                                     *
 *                                                                          *
 *   Free as in freedom.                                                    *
 ****************************************************************************/\n""")

include_libs = ("/********************************** BEGIN INCLUDE LIBS **********************************/\n")

for lib in ["stdio", "stdlib", "cblas"]:
    include_libs += (f"#include <{lib}.h>\n")

for l_lib in ["common"]:
    include_libs += (f"#include \"{l_lib}.h\"\n")

hardcoded_data = ("/********************************** BEGIN HARD CODED DATA **********************************/\n")

hardcoded_data += "#define LAYER0_SIZE INPUT_SIZE\n#define layer0_out input\n\n"

hardcoded_data += (f"#define INPUT_SIZE {len(MY_MODEL.layers[0].get_weights()[0])}\n")

idx = 0
for layer in MY_MODEL.layers:
    idx+=1

    # Este nos da la dimensión principal 'nrows' print(len(layer.get_weights()[0]))
    hardcoded_data += (f"#define LAYER{idx}_SIZE {len(layer.get_weights()[0][0])}\n") # este nos da la secundaria 'ncols'

    weight = "".join("".join(layer) for layer in np_array_to_hex(layer.get_weights()[0], FLOAT_LE))
    hardcoded_data += ( f"const fp32 * layer{idx}_weights = (fp32 *) \"" + r"\x" + r"\x".join(weight[i:i+2] for i in range(0, len(weight), 2)) + "\";\n" )

    bias = "".join("".join(layer) for layer in np_array_to_hex([layer.get_weights()[1]], FLOAT_LE))
    hardcoded_data += ( f"const fp32 * layer{idx}_bias = (fp32 *) \"" + r"\x" + r"\x".join(bias[i:i+2] for i in range(0, len(bias), 2)) + "\";\n" )


mallocs = ("/**************** BEGIN MALLOCS ****************/\n")
frees = ("/**************** BEGIN FREES ****************/\n")

# Mallocs and frees
idx = 0
for layer in MY_MODEL.layers:
    idx+=1
    mallocs += (f"fp32 * layer{idx}_out = malloc(input_dim * LAYER{idx}_SIZE * sizeof(fp32));\n")
    frees += (f"free(layer{idx}_out);\n")


algebra_ops = ("/**************** BEGIN ALGEBRA ****************/\n")


# Algebraic thinges
idx = 0
for layer in MY_MODEL.layers:
    idx+=1

    algebra_ops += f"fprintf(stderr, \"*** SGEMM %s x %s ***\\n\", \"layer{idx-1}\", \"layer{idx}\");\n"
    algebra_ops += f"cblas_sgemm(CblasRowMajor, CblasNoTrans, CblasNoTrans, input_dim, LAYER{idx}_SIZE, LAYER{idx-1}_SIZE, 1.f, (float *) layer{idx-1}_out, LAYER{idx-1}_SIZE, (float *) layer{idx}_weights, LAYER{idx}_SIZE, 0.f, (float *) layer{idx}_out, LAYER{idx}_SIZE);\n"
    # Una optimización muy sencilla es meter el free de layer{idx-1} después del sgemm, pero debido a que esto es una poc, no voy a complicarme ahora mismo, a no ser que sea necesario. 
    algebra_ops += f"fprintf(stderr, \"*** Map and Bias %s with %s function ***\\n\", \"layer{idx}_out\", \"{MAP_ALGORITHM}\");\n"
    algebra_ops += f"map_and_bias__fp32(layer{idx}_out, layer{idx}_bias, input_dim, LAYER{idx}_SIZE, 'N', {MAP_ALGORITHM});\n\n"

numlayers = len(MY_MODEL.layers)

# Print from template

print(f"""{header}

{include_libs}

{hardcoded_data}
"""+r"""
int main (int argc, char *argv[]) {

    if(argc < 2){
        fprintf(stderr, "Usage: %s input_file\n", argv[0]);
        exit(-1);
    }

    FILE *inputfile;
    fprintf(stderr, "*** Opening %s as input ***\n", argv[1]);
    if((inputfile = fopen(argv[1], "r")) == NULL){
        fprintf(stderr, "  -> Error: file %s does not exist\n", argv[1]);
        exit(-1);
    }

    // Parse input dims and allocate memory consquently
    int input_dim;
    fscanf(inputfile, "%d", &input_dim);
    
    fp32 * input;
    input = malloc(input_dim * INPUT_SIZE * sizeof(fp32));

    // Read input
    for(int i = 0; i < INPUT_SIZE * input_dim; i++){
        fscanf(inputfile, "%f,", &input[i].val);
    }

    // Finish reading input so let's close it
    fclose(inputfile);
"""+f"""
    {mallocs}

    {algebra_ops}
"""+r"""
    unsigned int greater_count = 0;
    for(int i = 0; i < input_dim; i++){
"""+f"""
        if(layer{numlayers}_out[i].val >= 0.5f) greater_count += 1;
"""+r"""
    }
    printf("\n\n--- SUMMARY ---\n");
    printf(" - Batch size = %d\n", input_dim);
    printf(" - Total >0.5 predictions = %d\n", greater_count);
    printf(" - %% >0.5 = %.2lf%%\n", ((double) greater_count / (double) input_dim)*100);
    printf("\n");
"""+f"""
    {frees}
    free(input);

    return 0;
}}""")


### Sparse code generation
Here on the other hand, we are generating C code for use with LibRSB, and we only use OpenBLAS' built-in ?omatcopy for transposing the input and output matrices. Given that, on top of the already installed AUR package `openblas-lapack`, we install, also from the AUR, the `librsb` package. Again, this is done on top of a fully up-to-date Arch Linux 64 bit installation, and with already fine tuned compilation `CFLAGS` in the `makepkg.conf`. On Ubuntu, `librsb-dev` is required too, but compiling it from source with help from the PKGBUILD from the AUR package is desirable. 

Same building environment as before, meaning that every file is located in the same folder. (Our freshly generated `.c` file, along with `common.{c,h}`)

For compilation, we re-use the previous command, while linking librsb, like so: `gcc -march=native -O3 -s *.c -o sparse.out -lm -lrsb -lcblas`.

If we are on Ubuntu, here we also change `-lcblas` by `-lopenblas`.

Some extra info on `librsb` compilation can be found at [their documentation webpage](http://librsb.sourceforge.net/#a_usage_c).

Generating the C code can be done with the Python code below. Same indications as the dense generation. The `MY_MODEL` constant is now particularly useful, as we can compare the computational characteristics of a fully dense implementation vs a sparse one by specifying `model_for_pruning` on the dense code.

In [ ]:
from datetime import date

MY_MODEL = model_for_pruning

MAP_ALGORITHM = "sigmoid__fp32"

header = (f"""/****************************************************************************
 * Copyright (C) {date.today().year} by Alonso Rodriguez                                   *
 *                                                                          *
 * This file is an auto-generated test.                                     *
 *                                                                          *
 *   Free as in freedom.                                                    *
 ****************************************************************************/\n""")

include_libs = ("/********************************** BEGIN INCLUDE LIBS **********************************/\n")

for lib in ["stdio", "stdlib", "rsb", "blas_sparse", "cblas"]:
    include_libs += (f"#include <{lib}.h>\n")

for l_lib in ["common"]:
    include_libs += (f"#include \"{l_lib}.h\"\n")

hardcoded_data = ("/********************************** BEGIN HARD CODED DATA **********************************/\n")

hardcoded_data += "#define LAYER0_SIZE INPUT_SIZE\n#define layer0_out input\n\n"

hardcoded_data += (f"#define INPUT_SIZE {len(MY_MODEL.layers[0].get_weights()[0])}\n")

for num_layer, layer in enumerate(MY_MODEL.layers,1):   
    nonzero_count = np.count_nonzero(layer.get_weights()[0])
    layer_num_weights = f'const fp32 * layer{num_layer}_weights = (fp32 *) \"'
    layer_num_nonzero = f'#define layer{num_layer}_nz {nonzero_count}'
    layer_num_idx_i = f'const int layer{num_layer}_i[layer{num_layer}_nz] = {{'
    layer_num_idx_j = f'const int layer{num_layer}_j[layer{num_layer}_nz] = {{'

    # Este nos da la dimensión principal 'nrows' print(len(layer.get_weights()[0]))
    hardcoded_data += (f"#define LAYER{num_layer}_SIZE {len(layer.get_weights()[0][0])}\n") # este nos da la secundaria 'ncols'
    
    for idx,value in enumerate(layer.get_weights()[0]):
        for item in np.nonzero(value)[0]:
            # print(f"{np_value_to_hex(value[item], FLOAT_BE)}, {idx}, {item}")
            temp_layer_num_weight = f'{np_value_to_hex(value[item], FLOAT_LE)}'
            layer_num_weights += r"\x" + r"\x".join(temp_layer_num_weight[i:i+2] for i in range(0, len(temp_layer_num_weight), 2))
            layer_num_idx_i   += f'{idx},'
            layer_num_idx_j   += f'{item},'

    layer_num_weights += '\";'
    layer_num_idx_i = layer_num_idx_i[:-1] + '};'
    layer_num_idx_j = layer_num_idx_j[:-1] + '};'

    hardcoded_data += f'{layer_num_nonzero}\n'
    hardcoded_data += f'{layer_num_weights}\n'
    hardcoded_data += f'{layer_num_idx_i}\n'
    hardcoded_data += f'{layer_num_idx_j}\n'

    bias = "".join("".join(layer) for layer in np_array_to_hex([layer.get_weights()[1]], FLOAT_LE))
    hardcoded_data += ( f"const fp32 * layer{num_layer}_bias = (fp32 *) \"" + r"\x" + r"\x".join(bias[i:i+2] for i in range(0, len(bias), 2)) + "\";\n" )


mallocs = ("/**************** BEGIN MALLOCS ****************/\n")
frees = ("/**************** BEGIN FREES ****************/\n")
matrix_creation = ("/**************** BEGIN MATRIX CREATION ****************/\n")

# Callocs and frees
for idx, layer in enumerate(MY_MODEL.layers, 1):
    mallocs += (f"blas_sparse_matrix layer{idx}_sp_weights = blas_invalid_handle;\n")
    mallocs += (f"fp32 * layer{idx}_out = calloc(input_dim * LAYER{idx}_SIZE, sizeof(fp32));\n")
    frees += (f"BLAS_usds(layer{idx}_sp_weights);\n")
    frees += (f"free(layer{idx}_out);\n")
    matrix_creation += (f"layer{idx}_sp_weights = BLAS_suscr_begin(LAYER{idx-1}_SIZE, LAYER{idx}_SIZE);\n")
    matrix_creation += (f"BLAS_suscr_insert_entries(layer{idx}_sp_weights, layer{idx}_nz, (float *) layer{idx}_weights, layer{idx}_i, layer{idx}_j);\n")
    matrix_creation += (f"BLAS_suscr_end(layer{idx}_sp_weights);\n")


algebra_ops = ("/**************** BEGIN ALGEBRA ****************/\n")


# Algebraic thinges
for idx, layer in enumerate(MY_MODEL.layers, 1):
    algebra_ops += f"fprintf(stderr, \"*** SUSMM %s(T) x %s(T) ***\\n\", \"layer{idx}\", \"layer{idx-1}\");\n"
    # algebra_ops += f"cblas_sgemm(CblasRowMajor, CblasNoTrans, CblasNoTrans, input_dim, LAYER{idx}_SIZE, LAYER{idx-1}_SIZE, 1.f, (float *) layer{idx-1}_out, LAYER{idx-1}_SIZE, (float *) layer{idx}_weights, LAYER{idx}_SIZE, 0.f, (float *) layer{idx}_out, LAYER{idx}_SIZE);\n"
    algebra_ops += f"BLAS_susmm(blas_rowmajor, blas_trans, input_dim, 1.f, layer{idx}_sp_weights, (float *) layer{idx-1}_out, input_dim, (float *) layer{idx}_out, input_dim);\n"

    # Una optimización muy sencilla es meter el free de layer{idx-1} después del sgemm, pero debido a que esto es una poc, no voy a complicarme ahora mismo, a no ser que sea necesario. 
    algebra_ops += f"fprintf(stderr, \"*** Map and Bias %s(T) with %s function ***\\n\", \"layer{idx}_out\", \"{MAP_ALGORITHM}\");\n"
    algebra_ops += f"map_and_bias__fp32(layer{idx}_out, layer{idx}_bias, LAYER{idx}_SIZE, input_dim, 'T', {MAP_ALGORITHM});\n\n"


numlayers = len(MY_MODEL.layers)
transpose_output = ("/*************** TRANSPOSE OUTPUT ***************/\n")
transpose_output += (f"#if LAYER{numlayers}_SIZE != 1\n")
transpose_output += (f"fp32 * temp_output = malloc(input_dim * LAYER{numlayers}_SIZE * sizeof(fp32));\n")
transpose_output += (f"fprintf(stderr, \"*** Transposing output out-of-place ***\\n\");\n")
transpose_output += (f"cblas_somatcopy(CblasRowMajor, CblasTrans, LAYER{numlayers}_SIZE, input_dim, 1.f, (float *) layer{numlayers}_out, input_dim, (float *) temp_output, LAYER{numlayers}_SIZE);\n")
transpose_output += (f"free(layer{numlayers}_out);\n")
transpose_output += (f"layer{numlayers}_out = temp_output;\n")
transpose_output += (f"temp_output = NULL;\n")
transpose_output += ("#endif\n")



# Print from template

print(f"""{header}

{include_libs}

{hardcoded_data}
"""+r"""
int main (int argc, char *argv[]) {

    if(argc < 2){
        fprintf(stderr, "Usage: %s input_file\n", argv[0]);
        exit(-1);
    }

    FILE *inputfile;
    fprintf(stderr, "*** Opening %s as input ***\n", argv[1]);
    if((inputfile = fopen(argv[1], "r")) == NULL){
        fprintf(stderr, "  -> Error: file %s does not exist\n", argv[1]);
        exit(-1);
    }

    // Parse input dims and allocate memory consquently
    int input_dim;
    fscanf(inputfile, "%d", &input_dim);
    
    fp32 * input;
    input = malloc(input_dim * INPUT_SIZE * sizeof(fp32));

    // Read input
    for(int i = 0; i < INPUT_SIZE * input_dim; i++){
        fscanf(inputfile, "%f,", &input[i].val);
    }

    // Finish reading input so let's close it
    fclose(inputfile);

    /*************** TRANSPOSE INPUT ***************/
#if LAYER0_SIZE != 1
    fp32 * temp_input = malloc(input_dim * INPUT_SIZE * sizeof(fp32));
    fprintf(stderr, "*** Transposing input out-of-place ***\n");
    cblas_somatcopy(CblasRowMajor, CblasTrans, input_dim, LAYER0_SIZE, 1.f, (float *) input, LAYER0_SIZE, (float *) temp_input, input_dim);
    free(input);
    input = temp_input;
    temp_input = NULL;
#endif
"""+f"""
    {mallocs}

    rsb_err_t errval = RSB_ERR_NO_ERROR;
    if( (errval = rsb_lib_init(RSB_NULL_INIT_OPTIONS)) != RSB_ERR_NO_ERROR ) return -1;

    {matrix_creation}

    {algebra_ops}

    {transpose_output}
"""+r"""
    unsigned int greater_count = 0;
    for(int i = 0; i < input_dim; i++){
"""+f"""
        if(layer{numlayers}_out[i].val >= 0.5f) greater_count += 1;
"""+r"""
    }

    printf("\n\n--- SUMMARY ---\n");
    printf(" - Batch size = %d\n", input_dim);
    printf(" - Total >0.5 predictions = %d\n", greater_count);
    printf(" - %% >0.5 = %.2lf%%\n", ((double) greater_count / (double) input_dim)*100);
    printf("\n");
"""+f"""
    {frees}
    free(input);

    if( (errval = rsb_lib_exit(RSB_NULL_EXIT_OPTIONS)) != RSB_ERR_NO_ERROR ) return -1;

    return 0;
}}""")
